# LDA Model Training

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Data

In [3]:
data_file_path='DATA.csv'
data=pd.read_csv(data_file_path)
data

,seq_no.,seq,module,motif_1,motif_2,motif_3,motif_4,motif_5,motif_string
0,0,GGAGGAGGAAGAGGCTGGGCCCCTGCTGTGTGGGGGCAAGTTCCCA...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3"
1,1,CAAATACCCTGGGGTGCAATACGACTTATATCTCACGTATTGGAAG...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4"
2,2,AACTAGGACACAGAAGTTGATCTAACGTAAACATCAAGAGCTTCCT...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4"
3,3,CACAGCTGGGCCTGGTTGGTCTTTGTCCAGGGAACAATGGAGCGCC...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4"
4,4,TTGTTTTATTTGTTTGTTGGGGGGCGGCGGGGAGCGACAGGGGAGT...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3"
...,...,...,...,...,...,...,...,...,...
995,995,CTATTATTAAGAAATATACACAATTTTAACTTCAAATATCTCTCAT...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3"
996,996,ATTGATTCTCACTTGCTTGACTCAAGGGAGGGTTTGATTTTGGTCA...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4"
997,997,ATGTGGTTCTACCATATAGTTTATCAATTTTAAACAGGTAAAATAT...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4"
998,998,ATTTTGTTGGTTAGGTGATGGAAGTATGATGCTATTGATATTTCCC...,2,0.0,1.0,0.0,0.0,1.0,"motif_2,motif_5"


In [4]:
print(data[data['motif_string'].isna()])
data.dropna(subset=['motif_string'],inplace=True)
data['motif_list']=data['motif_string'].apply(lambda x:x.split(','))
data

Empty DataFrame
Columns: [seq_no., seq, module, motif_1, motif_2, motif_3, motif_4, motif_5, motif_string]
Index: []


,seq_no.,seq,module,motif_1,motif_2,motif_3,motif_4,motif_5,motif_string,motif_list
0,0,GGAGGAGGAAGAGGCTGGGCCCCTGCTGTGTGGGGGCAAGTTCCCA...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3","[motif_2, motif_3]"
1,1,CAAATACCCTGGGGTGCAATACGACTTATATCTCACGTATTGGAAG...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4","[motif_1, motif_4]"
2,2,AACTAGGACACAGAAGTTGATCTAACGTAAACATCAAGAGCTTCCT...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4","[motif_1, motif_4]"
3,3,CACAGCTGGGCCTGGTTGGTCTTTGTCCAGGGAACAATGGAGCGCC...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4","[motif_1, motif_4]"
4,4,TTGTTTTATTTGTTTGTTGGGGGGCGGCGGGGAGCGACAGGGGAGT...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3","[motif_2, motif_3]"
...,...,...,...,...,...,...,...,...,...,...
995,995,CTATTATTAAGAAATATACACAATTTTAACTTCAAATATCTCTCAT...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3","[motif_2, motif_3]"
996,996,ATTGATTCTCACTTGCTTGACTCAAGGGAGGGTTTGATTTTGGTCA...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4","[motif_1, motif_4]"
997,997,ATGTGGTTCTACCATATAGTTTATCAATTTTAAACAGGTAAAATAT...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4","[motif_1, motif_4]"
998,998,ATTTTGTTGGTTAGGTGATGGAAGTATGATGCTATTGATATTTCCC...,2,0.0,1.0,0.0,0.0,1.0,"motif_2,motif_5","[motif_2, motif_5]"


In [5]:
from gensim.corpora import Dictionary

docs=data['motif_list'].values
dictionary=Dictionary(docs)

print('Motif-Index to Motif-Name Mapping:')
for i,v in dictionary.items():
    print(f'{i} - {v}')
    if i==10:
        break
    

Motif-Index to Motif-Name Mapping:
0 - motif_2
1 - motif_3
2 - motif_1
3 - motif_4
4 - motif_5


In [6]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('BOW (Sequence-0):')
docs[0],corpus[0]

BOW (Sequence-0):


(['motif_2', 'motif_3'], [(0, 1), (1, 1)])

## Training

In [7]:
# #setup logging for trainging metrics 
# import logging
# logging.basicConfig(filename='test_output/model_callbacks.log', filemode='w',
#                     format="%(asctime)s:%(levelname)s:%(message)s",
#                     level=logging.NOTSET)

# from gensim.models.callbacks import Callback,PerplexityMetric, ConvergenceMetric, CoherenceMetric
# perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
# convergence_logger = ConvergenceMetric(logger='shell')
# # coherence_cv_logger = CoherenceMetric(corpus=corpus, coherence = 'c_v', texts = docs)

# %%time
# from gensim.models import LdaModel,LdaMulticore

# #HYPERPARAMETERS
# #passes = epochs
# temp = dictionary[0]
# id2word = dictionary.id2token
# lda = LdaModel(corpus, id2word=id2word, alpha='auto',eval_every = 1,\
#                eta='auto',num_topics=3, iterations=5, passes = 10,
#               callbacks=[perplexity_logger,convergence_logger])

# lda.print_topics()

# %%time
# from gensim.models import LdaModel,LdaMulticore

# id2word = dictionary.id2token
# lda = LdaMulticore(corpus, id2word=id2word,eval_every = 1,\
#                eta='auto',num_topics=3, iterations=500, passes = 100)

# import pyLDAvis.gensim

# pyLDAvis.enable_notebook()
# pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [8]:
%%time
import logging
from gensim.models.callbacks import Callback,PerplexityMetric, ConvergenceMetric, CoherenceMetric

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
convergence_logger = ConvergenceMetric(logger='shell')
from gensim.models import LdaModel,LdaMulticore

temp = dictionary[0]
id2word = dictionary.id2token
lda = LdaModel(corpus, id2word=id2word, alpha='auto',chunksize=10000,
               eta='auto',num_topics=3, iterations=1000, passes = 5,
              minimum_probability=0.0,callbacks=[convergence_logger])

2021-02-22 04:53:31,950 : INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
2021-02-22 04:53:31,951 : INFO : using serial LDA version on this node
2021-02-22 04:53:31,952 : INFO : running online (multi-pass) LDA training, 3 topics, 5 passes over the supplied corpus of 1000 documents, updating model once every 1000 documents, evaluating perplexity every 1000 documents, iterating 1000x with a convergence threshold of 0.001000
2021-02-22 04:53:31,953 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2021-02-22 04:53:31,956 : DEBUG : bound: at document #0
2021-02-22 04:53:32,581 : INFO : -2.623 per-word bound, 6.2 perplexity estimate based on a held-out corpus of 1000 documents with 1931 words
2021-02-22 04:53:32,582 : INFO : PROGRESS: pass 0, at document #1000/1000
2021-02-22 04:53:32,583 : DEBUG : performing inference on a chunk of 1000 documents
2021-02-22 04:53:33,078 : DEB

CPU times: user 3.91 s, sys: 161 ms, total: 4.07 s
Wall time: 3.9 s


In [9]:
def getTopicDistribution(predictions,ntopics):
    topic_count_dict={i:0 for i in range(ntopics)}
    ndocs=len(predictions)
    for pred in predictions:
        top_topic=sorted(pred,key=lambda x:-x[1])[0][0]
        topic_count_dict[top_topic]+=1
    topic_dist_dict={k:v/ndocs for k,v in topic_count_dict.items()}
    return topic_dist_dict

def likelihoodMetric(predictions,ntopics):
    likelihood=0
    P_T=getTopicDistribution(predictions,ntopics)
#     print(P_T)
    for pred in tqdm(predictions):
        P_Xi_M=0
        for topic_no,P_Xi_T in pred:
            P_Xi_M+=P_Xi_T*P_T[topic_no]
        likelihood+=np.log10(P_Xi_M)
    print(likelihood)
    return likelihood

In [23]:
from gensim.models.coherencemodel import CoherenceModel

def coherenceMetric_cv(model,dictionary,docs):
    cm=CoherenceModel(model=model,dictionary=dictionary ,
                      texts=docs, coherence='c_v',processes=30,
                     window_size=2000)
    coherence = cm.get_coherence()
    print(coherence)
    return coherence
# coherenceMetric_cv(lda,dictionary ,docs)

In [21]:
from gensim.models.coherencemodel import CoherenceModel

def coherenceMetric_umass(model,dictionary,corpus):
    cm = CoherenceModel(model=model, corpus=corpus, \
                        coherence='u_mass',processes=30)
    coherence = cm.get_coherence()
    print(coherence)
    return coherence
# coherenceMetric_umass(lda,dictionary ,corpus)

In [25]:
from gensim.models.coherencemodel import CoherenceModel

def coherenceMetric_uci(model,dictionary,docs):
    cm=CoherenceModel(model=model,dictionary=dictionary ,
                      texts=docs, coherence='c_uci',processes=30,
                     window_size =2000)
    coherence = cm.get_coherence()
    print(coherence)
    return coherence
coherenceMetric_uci(lda,dictionary ,docs)

-12.312024693235896


-12.312024693235896

In [29]:
def perplexityMetric(model,corpus):
    perplexity=model.log_perplexity(corpus)
    print(perplexity)
    return perplexity

In [34]:
from sklearn.metrics.cluster import adjusted_rand_score

def randIndexMetric(predictions,data):
    pred_topic=[]
    for pred in tqdm(predictions):
        topic_prob=sorted(pred,key=lambda x:-x[1])
#         print(topic_prob)
        top_topic=topic_prob[0][0]
        pred_topic.append(top_topic)
    _data=data.copy()
    _data['pred_topic']=pred_topic
    ari=adjusted_rand_score(_data['module'], _data['pred_topic'])
    print(ari)
    return ari

In [26]:
from collections import Counter

def findTopMotifs(predictions,ntopics,data,ntop=5,outdir=None):
    pred_topic=[]
    for pred in tqdm(predictions):
        topic_prob=sorted(pred,key=lambda x:-x[1])
#         print(topic_prob)
        top_topic=topic_prob[0][0]
        pred_topic.append(top_topic)
    _data=data.copy()
    _data['pred_topic']=pred_topic      
    gb=_data[['motif_string','pred_topic']].groupby('pred_topic').\
    agg(lambda x: ','.join(x))
    gb['top_motif']=gb['motif_string'].\
    apply(lambda x:Counter(x.split(',')).most_common(ntop))
    gb.reset_index(inplace=True)
    gb=gb[['pred_topic','top_motif']]
    if outdir is not None:
        gb.to_csv(f'{outdir}/top{ntop}_motifs_topics_{ntopics}.csv',index=False)
    print(gb)
    return gb
# findTopMotifs(lda,corpus,data)

In [42]:
%%time
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from gensim.models import LdaModel,LdaMulticore

outdir='model_output'
eval_dict={'num_topics':[],'likelihood':[],'coherence_cv':[],\
          'coherence_umass':[],'coherence_uci':[],'perplexity':[],
          'ARI':[]}
temp = dictionary[0]
id2word = dictionary.id2token
for ntopics in range(2,10):
    print('\n'+'='*40)
    print('Num of Topics = '+str(ntopics))
    model = LdaModel(corpus, id2word=id2word, alpha='auto',chunksize=10000,
                   eta='auto',num_topics=ntopics, iterations=1000, passes = 5,
                  minimum_probability=0.0)
    
    predictions=model.get_document_topics(corpus,minimum_probability=0.0)
    print('\nFinding likelihood...')
    likelihood=likelihoodMetric(predictions,ntopics)
    print('\nFinding coherence_cv...')
    coherence_cv=coherenceMetric_cv(model,dictionary,docs)
    print('\nFinding coherence_umass...')
    coherence_umass=coherenceMetric_umass(model,dictionary ,corpus)
    print('\nFinding coherence_uci...')
    coherence_uci=coherenceMetric_uci(model,dictionary,docs)
    print('\nFinding perplexity...')
    perplexity=perplexityMetric(model,corpus)
    print('\nFinding ARI...')
    ari=randIndexMetric(predictions,data)
    print('\nFinding Top Motifs...')
    findTopMotifs(predictions,ntopics,data,outdir=outdir)
#     print('Findng avg. distance from TSS per topic...')
#     getAvgTssDist(predictions,ntopics,data,outdir=outdir)
    eval_dict['num_topics'].append(ntopics)
    eval_dict['likelihood'].append(likelihood)
    eval_dict['coherence_cv'].append(coherence_cv)
    eval_dict['coherence_umass'].append(coherence_umass)
    eval_dict['coherence_uci'].append(coherence_uci)
    eval_dict['perplexity'].append(perplexity)
    eval_dict['ARI'].append(ari)
eval_df=pd.DataFrame(eval_dict)
eval_df.to_csv(f'{outdir}/metrics.csv',index=False)
eval_df


Num of Topics = 2


  0%|          | 0/1000 [00:00<?, ?it/s]


Finding likelihood...


100%|██████████| 1000/1000 [00:00<00:00, 3626.61it/s]

-302.32415022479415

Finding coherence_cv...


...

100%|██████████| 1000/1000 [00:00<00:00, 5118.34it/s]

-1.85584503191353

Finding ARI...



100%|██████████| 1000/1000 [00:00<00:00, 5048.75it/s]

0.9236961587913273

Finding Top Motifs...
   pred_topic                                        top_motif
0           0                   [(motif_5, 42), (motif_2, 40)]
1           2    [(motif_1, 28), (motif_2, 28), (motif_3, 28)]
2           3  [(motif_2, 433), (motif_3, 433), (motif_1, 39)]
3           4    [(motif_1, 19), (motif_2, 19), (motif_3, 19)]
4           7                 [(motif_1, 478), (motif_4, 325)]
CPU times: user 31.2 s, sys: 2.83 s, total: 34 s
Wall time: 34.8 s


,num_topics,likelihood,coherence_cv,coherence_umass,coherence_uci,perplexity,ARI
0,2,-302.324150,0.220505,-13.967992,-12.312025,-1.473915,0.919285
1,3,-364.470659,0.220505,-13.767440,-12.312025,-1.496716,1.000000
2,4,-375.384129,0.220505,-13.804751,-12.312025,-1.488411,1.000000
3,5,-539.029680,0.220505,-13.809687,-12.312025,-1.703622,0.774275
4,6,-514.210145,0.220505,-13.857831,-12.312025,-1.713791,0.796370
5,7,-513.295313,0.220505,-13.932755,-12.312025,-1.804161,0.865497
6,8,-543.600288,0.220505,-13.917809,-12.312025,-1.857014,0.792390
7,9,-495.447391,0.220505,-13.770844,-12.312025,-1.855845,0.923696
